In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import datetime
import datetime

In [2]:
game_logistics_df = pd.read_csv('total_game_logistics.csv')
away_stats_df = pd.read_csv('total_away_stats.csv')
home_stats_df = pd.read_csv('total_home_stats.csv')
box_score_df = pd.read_csv('total_box_score_df.csv')

In [3]:
all_player_stats_df = away_stats_df.append(home_stats_df)

all_player_stats_df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)

# delete all rows with team totals
non_players = all_player_stats_df[(all_player_stats_df['Name'] == 'Team Totals')].index
all_player_stats_df.drop(non_players , inplace=True)

# delete all rows with players who did not play
benched_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Play')].index
all_player_stats_df.drop(benched_players , inplace=True)

# delete all rows with players who did not dress
out_players = all_player_stats_df[(all_player_stats_df['MP'] == 'Did Not Dress')].index
all_player_stats_df.drop(out_players , inplace=True)


In [4]:
all_player_stats_df = all_player_stats_df.fillna(0)
all_player_stats_df['Games'] = 1
all_player_stats_df.sort_values(by=['PTS'], ascending=False)

,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date,Games
267,De'Aaron Fox,SAC,33:43,14.0,25.0,0.560,2.0,5.0,0.400,7.0,...,5.0,2.0,0.0,0.0,2.0,5.0,37.0,-20.0,12/19/2022,1
274,Shai Gilgeous-Alexander,OKC,37:30,16.0,23.0,0.696,2.0,3.0,0.667,3.0,...,1.0,7.0,2.0,0.0,0.0,0.0,37.0,-10.0,1/20/2023,1
57,Nikola Vučević,ORL,31:22,15.0,23.0,0.652,2.0,5.0,0.400,4.0,...,13.0,0.0,2.0,0.0,3.0,1.0,36.0,12.0,11/17/2018,1
236,De'Aaron Fox,SAC,32:26,15.0,23.0,0.652,3.0,5.0,0.600,3.0,...,5.0,4.0,1.0,1.0,3.0,1.0,36.0,5.0,10/22/2022,1
70,Tim Hardaway Jr.,NYK,35:57,9.0,20.0,0.450,5.0,10.0,0.500,9.0,...,2.0,2.0,3.0,0.0,1.0,2.0,32.0,-24.0,11/18/2018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Malik Monk,SAC,13:03,0.0,4.0,0.000,0.0,2.0,0.000,0.0,...,2.0,1.0,1.0,0.0,2.0,1.0,0.0,-13.0,10/22/2022,1
180,Justin Robinson,SAC,2:52,0.0,3.0,0.000,0.0,1.0,0.000,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-3.0,12/19/2021,1
188,Daniel Theis,HOU,6:10,0.0,2.0,0.000,0.0,1.0,0.000,0.0,...,2.0,0.0,0.0,1.0,1.0,2.0,0.0,-1.0,1/14/2022,1
174,Tre Jones,SAS,10:20,0.0,0.0,0.000,0.0,0.0,0.000,0.0,...,3.0,4.0,0.0,0.0,1.0,0.0,0.0,8.0,12/19/2021,1


In [5]:
col_list = all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('Team')
col_list.remove('MP')
col_list.remove('FG%')
col_list.remove('3P%')
col_list.remove('FT%')
col_list.remove('Date')

total_sum_list = []

for col in col_list: 
    sum_list = []
    sum_list = all_player_stats_df.groupby('Name')[col].sum()
    total_sum_list.append(sum_list)
    
player_stat_totals_df = pd.DataFrame(total_sum_list)
player_stat_totals_df = player_stat_totals_df.transpose()

player_stat_totals_df = player_stat_totals_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

player_stat_totals_df['FG%'] = player_stat_totals_df.apply(lambda row: row.FG/row.FGA, axis = 1)
player_stat_totals_df['3P%'] = player_stat_totals_df.apply(lambda row: row.Three_P/row.Three_PA, axis = 1)
player_stat_totals_df['FT%'] = player_stat_totals_df.apply(lambda row: row.FT/row.FTA, axis = 1)

player_stat_totals_df = player_stat_totals_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

player_stat_totals_df = player_stat_totals_df.fillna(0)

player_stat_totals_df.loc[:, "FG%"] = player_stat_totals_df["FG%"].map('{:.3f}'.format)
player_stat_totals_df.loc[:, "3P%"] = player_stat_totals_df["3P%"].map('{:.3f}'.format)
player_stat_totals_df.loc[:, "FT%"] = player_stat_totals_df["FT%"].map('{:.3f}'.format)

FGP_Col = player_stat_totals_df.pop("FG%")
TPP_Col = player_stat_totals_df.pop("3P%")
FTP_Col = player_stat_totals_df.pop("FT%")

player_stat_totals_df.insert(6, 'FT%', FTP_Col)
player_stat_totals_df.insert(4, '3P%', TPP_Col)
player_stat_totals_df.insert(2, 'FG%', FGP_Col)

player_stat_totals_df

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_23335/1121349037.py:22: RuntimeWarning: invalid value encountered in double_scalars
  player_stat_totals_df['FG%'] = player_stat_totals_df.apply(lambda row: row.FG/row.FGA, axis = 1)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_23335/1121349037.py:23: RuntimeWarning: invalid value encountered in double_scalars
  player_stat_totals_df['3P%'] = player_stat_totals_df.apply(lambda row: row.Three_P/row.Three_PA, axis = 1)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_23335/1121349037.py:24: RuntimeWarning: invalid value encountered in double_scalars
  player_stat_totals_df['FT%'] = player_stat_totals_df.apply(lambda row: row.FT/row.FTA, axis = 1)


,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Games
Name,,,,,,,,,,,,,,,,,,,,
Aaron Gordon,23.0,41.0,0.561,6.0,13.0,0.462,5.0,8.0,0.625,7.0,10.0,17.0,8.0,4.0,1.0,5.0,5.0,57.0,29.0,3.0
Aaron Holiday,4.0,10.0,0.400,1.0,1.0,1.000,0.0,0.0,0.000,2.0,3.0,5.0,7.0,1.0,0.0,3.0,1.0,9.0,13.0,1.0
Al Horford,6.0,13.0,0.462,0.0,0.0,0.000,1.0,2.0,0.500,1.0,4.0,5.0,0.0,1.0,2.0,2.0,0.0,13.0,0.0,1.0
Alec Burks,11.0,19.0,0.579,2.0,4.0,0.500,10.0,15.0,0.667,2.0,3.0,5.0,3.0,1.0,0.0,2.0,5.0,34.0,8.0,2.0
Alex Len,11.0,18.0,0.611,0.0,2.0,0.000,1.0,6.0,0.167,6.0,15.0,21.0,7.0,2.0,11.0,5.0,12.0,23.0,8.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wes Iwundu,0.0,2.0,0.000,0.0,2.0,0.000,2.0,2.0,1.000,1.0,1.0,2.0,3.0,0.0,2.0,1.0,3.0,2.0,25.0,2.0
Wesley Johnson,7.0,14.0,0.500,1.0,3.0,0.333,0.0,0.0,0.000,0.0,6.0,6.0,0.0,0.0,0.0,2.0,3.0,15.0,-14.0,1.0
Xavier Munford,4.0,8.0,0.500,1.0,1.0,1.000,1.0,2.0,0.500,0.0,2.0,2.0,1.0,2.0,0.0,3.0,3.0,10.0,-5.0,1.0


In [6]:
player_stat_totals_df.sort_values(by=['Games'], ascending=False).head(50)

,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Games
Name,,,,,,,,,,,,,,,,,,,,
Harrison Barnes,45.0,109.0,0.413,9.0,37.0,0.243,50.0,65.0,0.769,9.0,41.0,50.0,33.0,8.0,1.0,17.0,17.0,149.0,-34.0,11.0
Davion Mitchell,32.0,92.0,0.348,10.0,32.0,0.312,10.0,14.0,0.714,5.0,14.0,19.0,35.0,5.0,2.0,13.0,18.0,84.0,-49.0,10.0
De'Aaron Fox,95.0,181.0,0.525,15.0,36.0,0.417,30.0,47.0,0.638,4.0,33.0,37.0,40.0,11.0,4.0,25.0,25.0,235.0,-41.0,10.0
Chimezie Metu,20.0,54.0,0.370,7.0,23.0,0.304,9.0,10.0,0.900,5.0,27.0,32.0,5.0,10.0,3.0,10.0,16.0,56.0,-40.0,9.0
Richaun Holmes,23.0,36.0,0.639,0.0,1.0,0.000,13.0,14.0,0.929,10.0,28.0,38.0,3.0,0.0,5.0,3.0,16.0,59.0,-31.0,7.0
Domantas Sabonis,51.0,88.0,0.580,4.0,12.0,0.333,25.0,38.0,0.658,23.0,72.0,95.0,45.0,10.0,3.0,19.0,27.0,131.0,16.0,7.0
Buddy Hield,38.0,90.0,0.422,23.0,59.0,0.390,6.0,8.0,0.750,7.0,14.0,21.0,12.0,5.0,2.0,8.0,9.0,105.0,-12.0,6.0
Tyrese Haliburton,38.0,70.0,0.543,12.0,29.0,0.414,10.0,12.0,0.833,6.0,17.0,23.0,33.0,16.0,3.0,10.0,15.0,98.0,-29.0,6.0
Damian Jones,21.0,36.0,0.583,3.0,6.0,0.500,7.0,11.0,0.636,13.0,13.0,26.0,2.0,2.0,4.0,4.0,7.0,52.0,-36.0,6.0


In [7]:
col_list = all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('MP')
col_list.remove('FG%')
col_list.remove('3P%')
col_list.remove('FT%')
col_list.remove('Date')

total_average_list = []

for col in col_list: 
    average_list = []
    teams_list = []
    if col == "Games":
        average_list = all_player_stats_df.groupby('Name')[col].sum()
        total_average_list.append(average_list)
    elif col == "Team":   
        
        #teams_list = all_player_stats_df.groupby('Name')[col]
        
        players_list = all_player_stats_df["Name"].values.tolist()
        teams_list = all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
#         first_teams_list = []
#         final_teams_list = []
#         [first_teams_list.append(item) for item in teams_list if item not in first_teams_list]
#         #new_teams_list = [*set(teams_list)]
#         #new_teams_list = list(set(teams_list))

#         for item in first_teams_list:
#             now_list = list(item)
#             final_teams_list.append(now_list)
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')
#         final_teams_list = new_teams_df["Team"].values.tolist()
        
#         teams_string = '/'.join(average_list)
#         total_average_list.append(final_teams_list)
    else:
        average_list = all_player_stats_df.groupby('Name')[col].mean()
        total_average_list.append(average_list)
    
temp_player_stat_average_df = pd.DataFrame(total_average_list)
temp_player_stat_average_df = temp_player_stat_average_df.transpose()
player_stat_average_df = pd.concat([temp_player_stat_average_df, new_teams_df], axis=1, join="inner")



player_stat_average_df = player_stat_average_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

player_stat_average_df['FG%'] = player_stat_average_df.apply(lambda row: row.FG/row.FGA, axis = 1)
player_stat_average_df['3P%'] = player_stat_average_df.apply(lambda row: row.Three_P/row.Three_PA, axis = 1)
player_stat_average_df['FT%'] = player_stat_average_df.apply(lambda row: row.FT/row.FTA, axis = 1)

player_stat_average_df = player_stat_average_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

player_stat_average_df = player_stat_average_df.fillna(0)

player_stat_average_df.loc[:, "FG%"] = player_stat_average_df["FG%"].map('{:.3f}'.format)
player_stat_average_df.loc[:, "3P%"] = player_stat_average_df["3P%"].map('{:.3f}'.format)
player_stat_average_df.loc[:, "FT%"] = player_stat_average_df["FT%"].map('{:.3f}'.format)

FGP_Col = player_stat_average_df.pop("FG%")
TPP_Col = player_stat_average_df.pop("3P%")
FTP_Col = player_stat_average_df.pop("FT%")

player_stat_average_df.insert(6, 'FT%', FTP_Col)
player_stat_average_df.insert(4, '3P%', TPP_Col)
player_stat_average_df.insert(2, 'FG%', FGP_Col)


player_stat_average_df = player_stat_average_df.round(2)

player_stat_average_df

SyntaxError: invalid syntax (4236394281.py, line 62)

In [ ]:
player_stat_average_df.sort_values(by=['PTS'], ascending=False).head(50)

In [ ]:
# val = all_player_stats_df['MP'].values[0]
# val2 = all_player_stats_df['MP'].values[1]

# datetime_object = datetime.datetime.strptime(val, '%M:%S').time()
# datetime_object2 = datetime.datetime.strptime(val2, '%M:%S').time()

# timeList = [val, val2]

# mysum = datetime.timedelta()
# for i in timeList:
#     (m, s) = i.split(':')
#     d = datetime.timedelta(minutes=int(m), seconds=int(s))
#     mysum += d
# print(str(mysum))

In [ ]:
player_stat_average_df

In [ ]:
print(all_player_stats_df.groupby('Name')['PTS'])

In [ ]:
l = ['SAC', 'ORL', 'SAC', 'ORL', 'SAC', 'ORL', 'SAC']
print("Original List: ", l)
res = [*set(l)]
print("List after removing duplicate elements: ", res)

In [ ]:
s = '/'.join(res)
print(s)